# Purpose

This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True

In [4]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import ta_lib.eda.api as eda

In [5]:
initialize_environment(debug=False, hide_warnings=True)

## Utility functions

### 1. Initialization

In [109]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
pprint(list_datasets(context))

['/raw/housing',
 '/cleaned/housing',
 '/train/housing/features',
 '/train/housing/target',
 '/test/housing/features',
 '/test/housing/target',
 '/score/sales/output']


In [7]:
housing_df = load_dataset(context, "raw/housing")

### 2. Data cleaning and consolidation

**<u>NOTES</u>**

The focus here is to create a cleaned dataset that is appropriate for solving the DS problem at hand from the raw data.

**1. Do**
* clean dataframe column names
* ensure dtypes are set properly
* join with other tables etc to create features
* transform, if appropriate, datetime like columns to generate additional features (weekday etc)
* transform, if appropriate, string columns to generate additional features
* discard cols that are not useful for training the model (IDs, constant cols, duplicate cols etc)
* additional features generated from existing columns


**2. Don't**
* handle missing values or outliers here. mark them and leave them for processing downstream.


#### 2.1 Clean individual tables 

#### Products Table

From data discovery, we know the following

* all columns are strings : nothing to fix. Apply generic cleaning (strip extra whitespace etc)
* ensure all `invalid` string entries are mapped to np.NaN
* some column are duplicates (eg. color, Ext_Color). Better to `coalesce` them instead of an outright discard of one of the columns.
* SKU is key column : ensure no duplicate values
* This will go into production code

In [26]:
housing_df_clean = (
    housing_df
    # generating a copy of the original DataFrame as a backup
    .copy()
    
    .replace({'': np.NaN})
    
    # clean column names
    .clean_names(case_type="snake")
)
housing_df_clean.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY


#### NOTE

It's always a good idea to save cleaned tabular data using a storage format that supports the following 

1. preserves the type information
2. language agnostic storage format
3. Supports compression
4. Supports customizing storage to optimize different data access patterns

For larger datasets, the last two points become crucial.

`Parquet` is one such file format that is very popular for storing tabular data. It has some nice properties:
- Similar to pickles & RDS datasets, but compatible with all languages
- Preserves the datatypes
- Compresses the data and reduces the filesize
- Good library support in Python and other languages
- As a columnar storage we can efficiently read fewer columns
- It also supports chunking data by groups of columns (for instance, by dates or a particular value of a key column) that makes loading subsets of the data fast.

In [44]:
save_dataset(context, housing_df_clean, 'cleaned/housing')

In [45]:
housing_df_clean.to_parquet("../../data/cleaned/housing.parquet")

### 3. Generate Train, Validation and Test datasets

- We split the data into train, test (optionally, also a validation dataset)
- In this example, we are binning the target into 10 quantiles and then use a Stratified Shuffle to split the data.
- See sklearn documentation on the various available splitters
- https://scikit-learn.org/stable/modules/classes.html#splitter-classes
- This will go into production code (training only)

In [101]:
from sklearn.model_selection import StratifiedShuffleSplit
from ta_lib.core.api import custom_train_test_split  # helper function to customize splitting
from scripts import *

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=context.random_seed)
housing_df_train, housing_df_test = custom_train_test_split(housing_df_clean, splitter, by=binned_median_income)

In [108]:
target_col = "median_house_value"

train_X, train_y = (
    housing_df_train
    .get_features_targets(target_column_names=target_col)
)

save_dataset(context, train_X, "train/housing/features")
save_dataset(context, train_y, "train/housing/target")

test_X, test_y = (
    housing_df_test
    .get_features_targets(target_column_names=target_col)
)

save_dataset(context, test_X, 'test/housing/features')
save_dataset(context, test_y, 'test/housing/target')